### Imports and image loading

In [1]:
import math

from PIL import Image
import numpy as np

In [2]:
cam = Image.open('cameraman.tif', mode='r')
tire = Image.open('tire.tif', mode='r')
lena = Image.open('lena.tiff', mode='r')

### Make a darker Lena for testing MSE and PSNR

In [3]:
lena_dark = np.array(lena) / 1.1
lena_dark = Image.fromarray(np.uint8(lena_dark))

In [4]:
def MSE(im1, im2):
    assert im1.size == im2.size and im1.mode == im2.mode

    SE = 0
    size = im1.size

    im1_data = np.array(im1).flatten()
    im2_data = np.array(im2).flatten()

    SE = sum(pow(im1_data - im2_data, 2))
    
    MSE = SE / (size[0] * size[1])
    return MSE

def PSNR(im1, im2):
    assert im1.size == im2.size and im1.mode == im2.mode
    mse = MSE(im1, im2)
    if mse == 0:
        return math.inf
    return 10 * math.log10(65025.0 / MSE(im1, im2)) # 255^2 = 65025

In [5]:
PSNR(lena, lena_dark)

23.380633641482028